In [1]:
import pandas as pd

In [2]:
%config Completer.use_jedi = False

In [3]:
from ast import literal_eval

In [43]:
from collections import Counter

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [6]:
df = pd.read_csv(r"D:\data\openparliament\df_climate.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
%%time
df['tokenized_content'] = df.tokenized_content.apply(literal_eval)

Wall time: 3.62 s


In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [9]:
%%time
df['tokens'] = df.text_en.apply(lambda x: simple_preprocess(x, deacc=True))

Wall time: 5.44 s


In [10]:
sentence = df.tokens.iloc[0]

In [39]:
def bigrams(sentence):
    part1 = [i+ "_"+j for i,j in zip(sentence[::2], sentence[1::2])]
    part2 = [i+ "_"+j for i,j in zip(sentence[1::2], sentence[2::2])]
    
    part1.extend(part2)
    return part1

In [19]:
a = ['a','b','c']
b = [1,2,3]

In [21]:
list(zip(a,b))

[('a', 1), ('b', 2), ('c', 3)]

In [41]:
df['bigrams'] = df.tokens.apply(bigrams)

Wall time: 442 ms


In [42]:
df.bigrams.head()

0    [mr_speaker, last_week, more_than, six_million...
1    [mr_speaker, last_week, more_than, six_million...
2    [mr_speaker, last_week, more_than, six_million...
3    [mr_speaker, it_is, privilege_to, be_able, to_...
4    [mr_speaker, it_is, privilege_to, be_able, to_...
Name: bigrams, dtype: object

In [62]:
bigram_counts = dict()
template_counter = Counter()
pids = list(set(df.politician_id))
for pol in pids:
    bigram_counts[pol] = template_counter.copy()

In [50]:
def update_counts(row):
    bigrams = row.loc['bigrams']
    if bigram_counts.get(row.loc['politician_id'])==None:
        bigram_counts[row.loc['politician_id']] = template_counter.copy()
        bigram_counts[row.loc['politician_id']].update(bigrams)
    else:
        bigram_counts[row.loc['politician_id']].update(bigrams)

In [57]:
row = df.iloc[0]

In [60]:
update_counts(row)

In [63]:
df.apply(update_counts,axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
19655    None
19656    None
19657    None
19658    None
19659    None
Length: 19660, dtype: object

In [66]:
bigram_counts[180].most_common(5)

[('in_the', 6),
 ('efforts_to', 6),
 ('across_the', 6),
 ('mr_speaker', 3),
 ('last_week', 3)]